<a href="https://colab.research.google.com/github/Ron-Rocks/Estimator/blob/master/Estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np 


In [13]:
mnist = tf.keras.datasets.mnist
(xTrain,yTrain),(xTest,yTest) = mnist.load_data()

xTrain = (xTrain.reshape(-1,784))/255
xTest = (xTest.reshape(-1,784))/255
 
print(yTrain)

[5 0 4 ... 5 6 8]


# From Keras

In [25]:
inputLayer = keras.Input(shape = (784,))
x = layers.Dense(64,activation = "relu")(inputLayer)
x =layers.Dense(128,activation = "relu")(x)
x = layers.Dense(64,activation = "relu")(x)
outputLayer = layers.Dense(10,activation = "softmax")(x)

Kerclassifier = keras.Model(inputs = inputLayer,outputs = outputLayer)


Kerclassifier.compile(loss = "sparse_categorical_crossentropy",optimizer = "adam",metrics = ["accuracy"])
Kerclassifier.fit(xTrain,yTrain,epochs = 20,validation_data = (xTest,yTest))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 5s 84us/sample - loss: 0.2635 - acc: 0.9210 - val_loss: 0.1422 - val_acc: 0.9552
Epoch 2/20
60000/60000 [==============================] - 5s 79us/sample - loss: 0.1126 - acc: 0.9659 - val_loss: 0.1041 - val_acc: 0.9660
Epoch 3/20
60000/60000 [==============================] - 5s 80us/sample - loss: 0.0835 - acc: 0.9739 - val_loss: 0.0857 - val_acc: 0.9723
Epoch 4/20
60000/60000 [==============================] - 5s 81us/sample - loss: 0.0676 - acc: 0.9782 - val_loss: 0.1033 - val_acc: 0.9688
Epoch 5/20
60000/60000 [==============================] - 5s 80us/sample - loss: 0.0552 - acc: 0.9825 - val_loss: 0.0933 - val_acc: 0.9708
Epoch 6/20
60000/60000 [==============================] - 5s 81us/sample - loss: 0.0468 - acc: 0.9847 - val_loss: 0.0926 - val_acc: 0.9718
Epoch 7/20
60000/60000 [==============================] - 5s 80us/sample - loss: 0.0391 - acc: 0.9871 - val_loss: 0.

# Now With Estimator

In [24]:
yTrain = yTrain.astype(int)
yTest = yTest.astype(int)
featuresNames = ["heightPixel","widthPixels"]

features = [tf.feature_column.numeric_column("x",shape = [28,28])]

classifier = tf.estimator.DNNClassifier(feature_columns=features,model_dir="model/MnistModel",n_classes=10,hidden_units=[64,128,64],activation_fn=tf.nn.relu)

trainInputFn = tf.estimator.inputs.numpy_input_fn(x = {"x":xTrain},y = yTrain,num_epochs=None,shuffle=True)
testInputFn = tf.estimator.inputs.numpy_input_fn(x = {"x":xTest},y = yTest,num_epochs=1,shuffle=False)

classifier.train(trainInputFn,steps=100000)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model/MnistModel', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd0237bcd68>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done c

# Now Evaluating both Models

In [27]:
EstEval = classifier.evaluate(input_fn=testInputFn)
KerEval = Kerclassifier.evaluate(xTest,yTest)

print("Results with Keras :  accuracy : ",KerEval[1]," loss :  ",KerEval[0])
print("Results with Estimator : ",EstEval)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-12-07T15:39:04Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/MnistModel/model.ckpt-102344
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-12-07-15:39:04
INFO:tensorflow:Saving dict for global step 102344: accuracy = 0.9664, average_loss = 0.32419384, global_step = 102344, loss = 41.037193
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 102344: model/MnistModel/model.ckpt-102344
10000/10000 [==============================] - 0s 32us/sample - loss: 0.1232 - acc: 0.9769
Results with Keras :  accuracy :  0.9769  loss :   0.12318351317809756
Results with Estimator :  {'accuracy': 0.9664, 'average_loss': 0.32419384, 'loss': 41.037193, 'global_step': 102344}


# Converting keras to Estimator

In [30]:
testInputFn = tf.estimator.inputs.numpy_input_fn(x = {"input_2":xTest},y = yTest,num_epochs=1,shuffle=False)

convertedModel = tf.keras.estimator.model_to_estimator(Kerclassifier)
ConvEval = convertedModel.evaluate(input_fn = testInputFn)


print("Converted model : ",ConvEval)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp2f9ihuby', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd0215b2358>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:ten